In [1]:
!unzip 0664343c9a8f11ea.zip

Archive:  0664343c9a8f11ea.zip
   creating: dataset/
   creating: dataset/test/
  inflating: dataset/test/508.jpg    
  inflating: dataset/test/246.jpg    
  inflating: dataset/test/473.jpg    
  inflating: dataset/test/485.jpg    
  inflating: dataset/test/128.jpg    
  inflating: dataset/test/410.jpg    
  inflating: dataset/test/465.jpg    
  inflating: dataset/test/196.jpg    
  inflating: dataset/test/340.jpg    
  inflating: dataset/test/467.jpg    
  inflating: dataset/test/306.jpg    
  inflating: dataset/test/171.jpg    
  inflating: dataset/test/323.jpg    
  inflating: dataset/test/332.jpg    
  inflating: dataset/test/330.jpg    
  inflating: dataset/test/170.jpg    
  inflating: dataset/test/479.jpg    
  inflating: dataset/test/9.jpg      
  inflating: dataset/test/283.jpg    
  inflating: dataset/test/114.jpg    
  inflating: dataset/test/507.jpg    
  inflating: dataset/test/183.jpg    
  inflating: dataset/test/200.jpg    
  inflating: dataset/test/457.jpg    
  inflat

In [2]:
import pandas as pd
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
print(train.head())

     Image         target
0   96.jpg       manipuri
1  163.jpg  bharatanatyam
2  450.jpg         odissi
3  219.jpg      kathakali
4  455.jpg         odissi


In [3]:
from fastai.imports import *
from fastai import *
from fastai.vision import *
from torchvision.models import *

In [32]:
path = 'dataset/'
test_path = path+'test/'
def generateSubmission(learn, string):
    id_list=list(test.Image)
    label=[]
    for iname in id_list:
        img=open_image(path+"test/"+iname)
        label.append(learn.predict(img)[0])
        if (len(label)%350 == 0):
            print(f'{len(label)} images done!')
    submissions = pd.DataFrame({'Image':id_list,'target':label})#, 'res': label_res, 'dense': label_dense})
    submissions.to_csv(string+"_submit.csv",index = False)
    return label

In [19]:
tfms = get_transforms(do_flip = True, max_lighting = 0.2, max_zoom= 1.1, max_warp = 0.15, max_rotate = 45)
data = ImageDataBunch.from_csv(path, folder= 'train/', 
                              valid_pct = 0.0,
                              csv_labels = 'train.csv',
                              ds_tfms = tfms, 
                              fn_col = 'Image',
                              #test = 'train_SOaYf6m/images', 
                              label_col = 'target',
                              bs = 16,
                              size = 300).normalize(imagenet_stats)

In [27]:
fbeta = FBeta(average='weighted', beta = 1)
learn = cnn_learner(data, models.resnet101, metrics=[accuracy, fbeta])

In [52]:
import warnings
warnings. filterwarnings('ignore')
learn.fit(epochs = 30, lr = 1e-3)

In [23]:
sub = pd.read_csv('submission.csv')
sub['res_101'] = generateSubmission(learn, 'res_101')
sub['res_101'] = sub['res_101'].astype('int')
print(sub.head())

     Image  target  res_101
0  150.jpg       0        6
1   48.jpg       0        5
2  440.jpg       0        6
3  302.jpg       0        6
4  323.jpg       0        5


In [53]:
del learn
learn1 = cnn_learner(data, models.resnet50, metrics=[accuracy, fbeta])
learn1.fit(epochs = 30, lr = 1e-3)

In [ ]:

learn2 = cnn_learner(data, models.densenet121, metrics=[accuracy, fbeta])
learn2.fit(epochs = 30, lr = 1e-3)

In [29]:

learn3 = cnn_learner(data, models.densenet161, metrics=[accuracy, fbeta])
learn3.fit(epochs = 30, lr = 1e-3)

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth


In [35]:
sub = pd.read_csv('submission.csv')
sub['res_101'] = generateSubmission(learn, 'res_101')
sub['res_101'] = sub['res_101'].astype('int')
print(sub.head())

     Image  target  res_101
0  150.jpg     NaN        3
1   48.jpg     NaN        7
2  440.jpg     NaN        3
3  302.jpg     NaN        3
4  323.jpg     NaN        3


In [34]:
FINAL_PATH='dataset/test'
def write__():
  with open('submission.csv',"w") as f:
    f.write('Image,target,model,model1,model2,model3,model4\n')
  for files in os.listdir(FINAL_PATH):
    with open('submission.csv',"a" ) as f:
        f.write(f'{files}\n')

write__()

In [51]:
path = 'dataset/'
test_path = path+'test/'
def generateSubmission():
    with open('submission.csv',"w") as f:
        f.write('Image,target,res_101,res_50,dense_121,dense_161,res_152\n')
    id_list=list(test.Image)
    label=[]
    for iname in id_list:
        img=open_image(path+"test/"+iname)
        label=learn.predict(img)[0]
        
        with open('submission.csv','a') as f:
            f.write(f'{iname},{learn.predict(img)[0]},{learn1.predict(img)[0]},{learn2.predict(img)[0]},{learn3.predict(img)[0]},{learn4.predict(img)[0]}\n')
    
generateSubmission(learn2)

In [41]:
learn4 = cnn_learner(data, models.resnet152, metrics=[accuracy, fbeta])
learn4.fit(epochs = 30, lr = 1e-3)

Downloading: "https://download.pytorch.org/models/resnet152-b121ed2d.pth" to /root/.cache/torch/checkpoints/resnet152-b121ed2d.pth
